In [4]:
import os
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
import regex as re
import requests
import json

pd.set_option('display.max_rows', None)
pd.set_option('mode.chained_assignment',None)

In [5]:
headers = {
    'authority': 'www.zillow.com',
    'sec-ch-ua': '"Google Chrome";v="89", "Chromium";v="89", ";Not A Brand";v="99"',
    'sec-ch-ua-mobile': '?0',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.114 Safari/537.36',
    'accept': '*/*',
    'sec-fetch-site': 'same-origin',
    'sec-fetch-mode': 'cors',
    'sec-fetch-dest': 'empty',
    'referer': 'https://www.zillow.com/co/?searchQueryState=%7B%22usersSearchTerm%22%3A%22CO%22%2C%22mapBounds%22%3A%7B%22west%22%3A-111.735869734375%2C%22east%22%3A-99.365264265625%2C%22south%22%3A36.269455423479094%2C%22north%22%3A41.67982549177817%7D%2C%22mapZoom%22%3A7%2C%22regionSelection%22%3A%5B%7B%22regionId%22%3A10%2C%22regionType%22%3A2%7D%5D%2C%22isMapVisible%22%3Atrue%2C%22filterState%22%3A%7B%22built%22%3A%7B%22max%22%3A2010%7D%2C%22att%22%3A%7B%22value%22%3A%22new%22%7D%2C%22ah%22%3A%7B%22value%22%3Atrue%7D%7D%2C%22isListVisible%22%3Atrue%7D',
    'accept-language': 'en-US,en;q=0.9',
}
list1=['new','fresh%20paint','new%20premium','quartz','remodeled','renovated','shiplap','soft%20close','staging','stainless','turn%20key','updated']
state='WA'

In [6]:
for j in list1:
    with requests.session() as s:
        page = 1
        end_page = 20
        url = ''
        url_list = []
    
        while page <= end_page:
            url = 'https://www.zillow.com/co/4_p/?searchQueryState=%7B%22usersSearchTerm%22%3A%22'+state+'%22%2C%22mapBounds%22%3A%7B%22west%22%3A-111.735869734375%2C%22east%22%3A-99.365264265625%2C%22south%22%3A36.269455423479094%2C%22north%22%3A41.67982549177817%7D%2C%22mapZoom%22%3A7%2C%22regionSelection%22%3A%5B%7B%22regionId%22%3A10%2C%22regionType%22%3A2%7D%5D%2C%22isMapVisible%22%3Atrue%2C%22filterState%22%3A%7B%22built%22%3A%7B%22max%22%3A2010%7D%2C%22att%22%3A%7B%22value%22%3A%22'+j+'%22%7D%2C%22ah%22%3A%7B%22value%22%3Atrue%7D%7D%2C%22isListVisible%22%3Atrue%2C%22pagination%22%3A%7B%22currentPage%22%3A'+str(page)+'%7D%7D'
            url_list.append(url)
            page += 1
    
    request = ''
    request_list = []
    
    for url in url_list:
        request = s.get(url, headers=headers)
        request_list.append(request)
    
    soup = ''
    soup_list = []

    for request in request_list:
        soup = BeautifulSoup(request.content, 'lxml')
        soup_list.append(soup)
    df_list = []
    for soup in soup_list:
        df = pd.DataFrame()
        for i in soup:
            address = (soup.find_all (class_= 'list-card-addr'))
            price = (soup.find_all (class_='list-card-price'))
            beds = (soup.find_all("ul", class_="list-card-details"))
            link = soup.find_all (class_= 'list-card-link')

            df['prices'] = price
            df['address'] = address
            df['beds'] = beds
            
        urls = []
        
        for link in soup.find_all("article"):
            try:
                href = link.find('a',class_="list-card-link")['href']
            except Exception as e:
                href=''
            #raise e 

            urls.append(href)
        if '' in urls:
            urls.remove('')
        df['links'] = urls
        df['links'] = df['links'].astype('str')
        df_list.append(df)
    df = pd.concat(df_list).reset_index().drop(columns='index')
    df['address'] = df['address'].astype('str')
    df['beds'] = df['beds'].astype('str')
    df.loc[:,'address'] = df.loc[:,'address'].replace('<address class="list-card-addr">', '', regex=True)

    df.loc[:,'address'] = df.loc[:,'address'].replace('</address>', '', regex=True)
    #df['prices'] = df['prices'].str.replace(r'\D', '')

    #filter unwanted property types
    df = df[~df['beds'].str.contains("Land for sale")]

    #remove html tags from beds column
    df.loc[:,'beds'] = df.loc[:,'beds'].replace('<ul class="list-card-details"><li class="">', ' ', regex=True)
    df.loc[:,'beds'] = df.loc[:,'beds'].replace('<abbr class="list-card-label"> <!-- -->bds</abbr></li><li class="">', ' ', regex=True)
    df.loc[:,'beds'] = df.loc[:,'beds'].replace('<abbr class="list-card-label"> <!-- -->ba</abbr></li><li class="">', ' ', regex=True)
    df.loc[:,'beds'] = df.loc[:,'beds'].replace('<abbr class="list-card-label"> <!-- -->bd</abbr></li><li class="">', ' ', regex=True)
    df.loc[:,'beds'] = df.loc[:,'beds'].replace('-<abbr class="list-card-label"> <!-- -->Foreclosure</abbr>', '- Foreclosure', regex=True)
    df.loc[:,'beds'] = df.loc[:,'beds'].replace('<abbr class="list-card-label"> <!-- -->sqft</abbr></li><li class="list-card-statusText">', ' ', regex=True)
    df.loc[:,'beds'] = df.loc[:,'beds'].replace('<abbr class="list-card-label"> <!-- -->acres lot</abbr></li><li class="list-card-statusText">', ' ', regex=True)
    df.loc[:,'beds'] = df.loc[:,'beds'].replace('</li></ul>', '', regex=True)
    df.loc[:,'beds'] = df.loc[:,'beds'].replace('--', '0', regex=True)
    df.loc[:,'beds'] = df.loc[:,'beds'].replace('Multi-family', 'Multifamily', regex=True)
    df.loc[:,'beds'] = df.loc[:,'beds'].replace(' for sale', '', regex=True)
    df.loc[:,'beds'] = df.loc[:,'beds'].replace('-<abbr class="list-card-label"> <!0 0>Auction</abbr>', '- Auction', regex=True)
    df.loc[:,'beds'] = df.loc[:,'beds'].replace('-<abbr class="list-card-label"> <!0 0>Pending</abbr>', '- Pending', regex=True)
    df[['beds','type']] = df.beds.apply(lambda x: pd.Series(str(x).split('-')))
    df[['beds', 'baths', 'sq_feet']] = df.beds.str.split(expand=True)
    df['prices']=df.prices.apply(lambda x: pd.Series(str(x).split('>')[1].split('<')[0]))
    urls=df['links']
    urls1=urls.tolist()
    agent=[]
    contact=[]
    agentComp=[]
    des=[]
    for i in urls:
        headers = {
        'authority': 'www.zillow.com',
        'sec-ch-ua': '"Google Chrome";v="89", "Chromium";v="89", ";Not A Brand";v="99"',
        'sec-ch-ua-mobile': '?0',
        'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.114 Safari/537.36',
        'accept': '*/*',
        'sec-fetch-site': 'same-origin',
        'sec-fetch-mode': 'cors',
        'sec-fetch-dest': 'empty',
        'referer': i,
        'accept-language': 'en-US,en;q=0.9',
        }
        response=requests.get(i,headers=headers)
        soup=BeautifulSoup(response.content,'lxml')

        data = json.loads(soup.find_all('script', type='application/json')[2].string)
        data1=json.loads(data['apiCache'])
        l1=list(data1.keys())
        a=(str(l1[1]))
        data2=data1[a]['property']['attributionInfo']
        agent.append(data2['infoString4'])
        contact.append(data2['infoString5'])
        agentComp.append(data2['infoString6'])
        des.append(data1[a]['property']['description'])
    df['Listing Agent Name']=agent
    df['Listing Agent Contact']=contact
    df['Listing Agent Company']=agentComp
    df['Description']=des
    df['Key Word']=j
    df.to_csv(state+'-'+j+'.csv')
    print('done')
            # <p data-testid="attribution-broker" class="Text-c11n-8-62-4__sc-aiai24-0 bnmVPu">Keller Williams Real Estate Services 818-432-3200</p>


IndexError: list index out of range